In [ ]:
# Install required packages (run manually before running the script)
# pip install langchain openai wikipedia duckduckgo-search beautifulsoup4 unstructured requests

from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, Tool
import os

# Create custom tools
wikipedia_tool = WikipediaQueryRun()
duckduckgo_tool = DuckDuckGoSearchRun()

# Web scraping function using WebBaseLoader
def web_scraper(url: str) -> str:
    try:
        loader = WebBaseLoader(url)
        docs = loader.load()
        return docs[0].page_content
    except Exception as e:
        return f"Web scraping failed: {e}"

# Wrap the web scraper as a LangChain tool
web_scraper_tool = Tool(
    name="WebScraper",
    func=web_scraper,
    description="Extract content from a website URL"
)

# Initialize the agent
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
tools = [
    Tool(name="Wikipedia", func=wikipedia_tool.run, description="Search Wikipedia"),
    Tool(name="DuckDuckGo", func=duckduckgo_tool.run, description="Search DuckDuckGo for recent information"),
    web_scraper_tool
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Execute the research query
query = "Research about the XZ backdoor"
result = agent.run(query)

# Save research output to a text file
filename = "research_output.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write(result)

print(f"Research is complete and saved to '{filename}'.")
